In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle
import time
import random
import json
import os
import glob
from tqdm import tqdm

## Scrape Species List

In [ ]:
URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

BOWlist = soup.find_all('a')

URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

'''
# Create a list with links
BOWpages = ['https://birdsoftheworld.org' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

'''
# Create a list with links
BOWpages = ['https://birdsoftheworld-org.ezproxy.library.wur.nl' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

## Open Driver (Selenium)

In [ ]:
# Init driver
browser = webdriver.Safari()

time.sleep(5)

# Login URL
browser.get("https://edepot.wur.nl/541124")

time.sleep(8)

# Current page
main_page = browser.current_window_handle

# Hit Okay
Ok = browser.find_element_by_id("loginlinkBtn")
Ok.send_keys(u'\ue007')

time.sleep(3)

# changing the handles to access login page
for handle in browser.window_handles:
    if handle != main_page:
        login_page = handle
          
# change the control to signin page        
browser.switch_to.window(login_page)

# Username
username = browser.find_element_by_id("userNameInput")
username.clear()
username.send_keys("robert.vandevlasakker@wur.nl")

# Password
password = browser.find_element_by_name("Password")
password.clear()
password.send_keys("XXXXXX")

# Hit Okay
Ok = browser.find_element_by_id("submitButton")
Ok.send_keys(u'\ue007')

time.sleep(8)

'''
# changing the handles to access login page
for handle in browser.window_handles:
    if handle != main_page or handle!= login_page:
        BOW = handle
'''

# change the control to signin page        
browser.switch_to.window(main_page)

time.sleep(5)

In [ ]:
# Possible pages
Pages = ['Introduction', 'Appearance', 'Systematics', 'Distribution',
         'Habitat', 'Movement', 'Foodhabits', 'Sounds',
         'Behavior', 'Breeding', 'Demography', 'Conservation',
         'Other', 'Priorities']

for BirdWebPage in BOWpages[8000:]:
    
    # Init page
    BirdURL = BirdWebPage

    # Get the bird page
    browser.get(BirdURL)

    # Set a sleep timer
    SleepTime = random.randint(3, 4)

    # Read the XML file
    HTML = browser.page_source

    # Species name
    soup = BeautifulSoup(HTML, 'html.parser')
    SpeciesName = soup.title.text.split(' - ')[0].lstrip()

    # Create folder
    os.mkdir('../data/raw/' + SpeciesName)

    # Dump the HTML file
    with open('../data/raw/' + SpeciesName + '/' + Pages[0] + '.html', "w") as f:
          f.write(HTML)

    # Check URL for shorter account
    URL_intro = browser.current_url

    # Sleep Just in case
    time.sleep(SleepTime)

    # Loop over possible pages
    for Index, Page in enumerate(Pages[1:]):

        # Create URL
        URL = BirdURL + Page.lower()
        # Go to URL
        browser.get(URL)

        # Get current URL
        URL_check = browser.current_url

        # Check if short account
        if URL_check == URL_intro and Index < 2:
            time.sleep(4)
            break
        # Check if links are missing    
        elif URL_check == URL_intro and Index > 2:
            time.sleep(4)
            continue            

        # Get the HTML info 
        HTML = browser.page_source

        # Dump the HTML file
        with open('../data/raw/' + SpeciesName + '/' + Page + '.html', "w") as f:
              f.write(HTML)

        # Sleep 
        time.sleep(4)

In [ ]:
# Possible pages
Pages = ['Introduction', 'Appearance', 'Systematics', 'Distribution',
         'Habitat', 'Movement', 'Foodhabits', 'Sounds',
         'Behavior', 'Breeding', 'Demography', 'Conservation',
         'Other', 'Priorities']

Testpages = ['https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/species/bertin1/cur/',
             'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/species/norcar/cur/',
             'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/species/ostric2/cur/']

for BirdWebPage in Testpages:
    
    # Init page
    BirdURL = BirdWebPage

    # Get the bird page
    browser.get(BirdURL)

    # Set a sleep timer
    SleepTime = random.randint(3, 4)

    # Read the XML file
    HTML = browser.page_source

    # Species name
    soup = BeautifulSoup(HTML, 'html.parser')
    SpeciesName = soup.title.text.split(' - ')[0].lstrip()

    # Create folder
    os.mkdir('../data/raw/' + SpeciesName)

    # Dump the HTML file
    with open('../data/raw/' + SpeciesName + '/' + Pages[0] + '.html', "w") as f:
          f.write(HTML)

    # Check URL for shorter account
    URL_intro = browser.current_url
    print(URL_intro)

    # Sleep Just in case
    time.sleep(SleepTime)

    # Loop over possible pages
    for Index, Page in enumerate(Pages[1:]):

        # Create URL
        URL = BirdURL + Page.lower()
        # Go to URL
        browser.get(URL)

        # Get current URL
        URL_check = browser.current_url
        print(URL_check)

        # Check if short account
        if URL_check == URL_intro and Index < 2:
            time.sleep(4)
            break
        # Check if links are missing    
        elif URL_check == URL_intro and Index > 2:
            time.sleep(4)
            continue            

        # Get the HTML info 
        HTML = browser.page_source

        # Dump the HTML file
        with open('../data/raw/' + SpeciesName + '/' + Page + '.html', "w") as f:
              f.write(HTML)

        # Sleep 
        time.sleep(4)

# Parser

In [ ]:
SpeciesList = glob.glob('../../data/raw/*')
SingleList = [Species + '/Introduction.html' for Species in SpeciesList if len(glob.glob(Species + '/*')) == 1]

DataBOW = collections.defaultdict(list)

SourceRemover = [
    ' \(\d+.+?Close\n\t\n\)'
]

for BirdXML in SingleList[0:]:
    with open(BirdXML) as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    # Birdname
    BirdName = soup.title.text.split(' - ')[0].lstrip()
    
    # Chapters
    Paragraphs = soup.find_all('p')
    
    for Para in Paragraphs:
        try:
            Chapter = Para.find_previous_sibling().find('h2').text
            
            # Clean text                  
            for Remover in SourceRemover:
                TextCleaned = re.sub(Remover, '', Para.text, flags=re.DOTALL)
            TextCleaned = TextCleaned.replace('\n', '') 
            
            if Chapter == 'Identification':
                DataBOW[BirdName].append(tuple([1, TextCleaned]))
            else:
                DataBOW[BirdName].append(tuple([0, TextCleaned]))
        except:
            continue
            
            
with open('../../data/processed/dataBOW.pkl', 'wb') as f:
    pickle.dump(DataBOW, f)

## BOW Description data

In [ ]:
# Read all html files
species_folder = glob.glob('../../data/raw/BOW/*')
multi_list = [Species + '/Appearance.html' for Species in species_folder if len(glob.glob(Species + '/*')) != 1]
single_list = [Species + '/Introduction.html' for Species in species_folder if len(glob.glob(Species + '/*')) == 1]

In [ ]:
for html in tqdm(multi_list[0:2]):
    # Open the html
    with open(html) as f:
        # Structure it
        soup = BeautifulSoup(f, 'html.parser')
        try:
            # Extract name
            name = soup.title.text.strip().split(' - ')[2]
            # Spans
            spans = [span.text for span in soup.find_all('p') if not 'fig' in span.text]
            # Append
            data_bow[name] = spans
        except:
            continue

In [ ]:
soup.title.text.strip().split(' - ')[2]

In [ ]:
# Cleaned data
data_bow = collections.defaultdict(list)

for html in tqdm(multi_list):
    # Open the html
    with open(html) as f:
        # Structure it
        soup = BeautifulSoup(f, 'html.parser')
        try:
            # Extract name
            name = soup.title.text.strip().split(' - ')[2]
            # Spans
            spans = [span.text for span in soup.find_all('p') if not 'fig' in span.text]
            # Append
            data_bow[name] = spans
        except:
            continue
    
for html in tqdm(single_list):    
    # Open the html
    with open(html) as f:
        # Structure it
        soup = BeautifulSoup(f, 'html.parser')
        # Extract name
        name = soup.title.text.strip().split(' - ')[1]
        # Spans
        for span in soup.find_all('p'):
            #print(span.text)
            try:
                chapter = span.find_previous_sibling().find('h2').text
                #print(chapter)
                # Check if part of Description chapter
                if chapter == 'Identification':
                    data_bow[name] = span.text
            except:
                continue
                
with open('../data/processed/descriptions_birds_bow.pkl', 'wb') as f:
    pickle.dump(data_bow, f)

# BOW Sentences per Species

In [1]:
import numpy as np
import pandas as pd
import torch
import pickle
import re
import requests
from matplotlib import cm
import matplotlib
from bs4 import BeautifulSoup
import collections
from itertools import chain
from collections import Counter
import torch.nn as nn
import glob
import random
from pathlib import Path
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, Dataset, random_split
from tqdm import tqdm
import time
import urllib.parse

import spacy
from spacy import displacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.util import filter_spans
nlp = spacy.load("en_core_web_lg")

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

import sys
sys.path.insert(0, '../../src/models/')
sys.path.insert(0, '../../src/features/')
#sys.path.insert(0, '../src/visualization/')

import predict_model
from build_features import text_cleaner
#import visualize as vis

%matplotlib inline

In [2]:
model = predict_model.loadBERT("../../models/", 'saved_weights_inf_FIXED_boot.pt')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


def SpanPredictor(span, pred_values=False):
    
    """
    Uses a trained bert classifier to see if a span
    belongs to a species description or otherwise.
    """
         
    with torch.no_grad():
        # Tokenize input
        inputs = tokenizer(span, return_tensors="pt", truncation=True)
        # Predict class
        outputs = model(**inputs)
        # Get prediction values
        exps = torch.exp(outputs)
        # Get class
        span_class = exps.argmax(1).item()

        # Print the prediction values
        if pred_values:
            return span_class, exps[0]
        else:
            return span_class

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


CPU Success


In [3]:
# Read all html files
species_folder = glob.glob('../../data/raw/BOW/*')
multi_list = [Species + '/Appearance.html' for Species in species_folder if len(glob.glob(Species + '/*')) != 1]
single_list = [Species + '/Introduction.html' for Species in species_folder if len(glob.glob(Species + '/*')) == 1]

In [4]:
data = collections.defaultdict(list)

for html in tqdm(single_list):
    with open(html) as f:
        # Structure it
        soup = BeautifulSoup(f, 'html.parser')
        # Extract name
        species = soup.title.text.strip().split(' - ')[1]
        # Find all non Identification Spans
        spans = [span for span in soup.find_all('p') 
                 if not 'fig' in span.text]
        for span in spans:
            if span.find_previous_sibling() != None:
                if span.find_previous_sibling().find('h2') != None:
                    # Locate Identification
                    if span.find_previous_sibling().find('h2').text == 'Identification':
                        text_id = span.text.strip().replace('\n', "").replace('; ', '. ')
                        text_id = re.sub(' +',' ', text_id)
                        text_list = text_id.split('. ')
                        for sentence in text_list:
                            if SpanPredictor(sentence):
                                #print(URL)
                                data[species].append(sentence)
                    else:
                        sentences = text_cleaner(span.text)
                        # Loop over the individual sentences
                        for sentence in sentences:                    
                            # Create string object
                            sentence_str = str(sentence)
                            #print(sentence_str)

                            if SpanPredictor(sentence_str):
                                #print(URL)
                                data[species].append(sentence_str)
                else:
                    sentences = text_cleaner(span.text)
                    # Loop over the individual sentences
                    for sentence in sentences:                    
                        # Create string object
                        sentence_str = str(sentence)
                        #print(sentence_str)

                        if SpanPredictor(sentence_str):
                            #print(URL)
                            data[species].append(sentence_str)
            else:
                continue

100%|█████████████████████████████████████| 9090/9090 [5:01:34<00:00,  1.99s/it]


In [5]:
with open('../../data/processed/scrapeddata_train_species_description_bow.pkl', 'wb') as f:
    pickle.dump(data, f)

In [9]:
data['Sylvietta leucophrys']

['Juvenile has head to throat, breast and flanks dark brown, supercilium tinged yellowish, underparts pale yellow.',
 'Race chloronota is greener above than nominate, with more extensive chestnut-brown on cheeks and ear-coverts.']

In [ ]:
data

In [ ]:
dirty_text